# 03 Repeatability / 再現性評価

Compare paired captures of the same tire, compute RMSE and correlation, and visualize differences.

In [ ]:
# Ensure the repository root is importable (VS Code / Windows friendly)
import sys
from pathlib import Path

NOTEBOOK_ROOT = Path().resolve()
repo_root = None
for candidate in [NOTEBOOK_ROOT, NOTEBOOK_ROOT.parent, NOTEBOOK_ROOT.parent.parent]:
    if (candidate / "tire_profiler").exists():
        repo_root = candidate
        break

if repo_root is None:
    raise RuntimeError("Unable to locate tire_profiler package relative to this notebook")

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tire_profiler.align import align_points
from tire_profiler.cylinder import fit_cylinder
from tire_profiler.rimline import fit_rimline
from tire_profiler.slice_profile import cylindrical_features, slice_band, compute_profile

In [ ]:
import plotly.graph_objects as go

def preview_cloud(points, color=None, title="Point cloud preview", sample=20000):
    points = np.asarray(points)
    if points.size == 0:
        raise ValueError("No points available for visualization")
    idx = None
    if len(points) > sample:
        rng = np.random.default_rng(0)
        idx = rng.choice(len(points), sample, replace=False)
    pts = points if idx is None else points[idx]
    if color is None:
        color_vals = pts[:, 2]
    else:
        color = np.asarray(color)
        color_vals = color if idx is None else color[idx]
    fig = go.Figure(data=[go.Scatter3d(
        x=pts[:, 0],
        y=pts[:, 1],
        z=pts[:, 2],
        mode='markers',
        marker=dict(size=2, color=color_vals, colorscale='Viridis', opacity=0.85)
    )])
    fig.update_layout(
        title=title,
        scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z', aspectmode='data'),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.show()


In [ ]:
def synthetic_tire_points(seed: int, capture: int, n: int = 110_000):
    rng = np.random.default_rng(seed + capture * 17)
    x = rng.uniform(-0.12, 0.12, n)
    theta = rng.uniform(-np.pi, np.pi, n)
    radius = 0.34 + 0.004 * np.sin(5 * theta + 0.1 * capture)
    y = radius * np.sin(theta)
    z = radius * np.cos(theta)
    z += 0.002 * capture * np.sin(theta)
    rim_mask = np.abs(x) > 0.095
    z[rim_mask] += 0.005 * np.exp(-((theta[rim_mask]) ** 2) / 0.09)
    pts = np.stack([x, y, z], axis=1) + rng.normal(scale=0.0006, size=(n, 3))
    return pts

In [ ]:
sample_a = synthetic_tire_points(seed=0, capture=0)
sample_b = synthetic_tire_points(seed=0, capture=1)
preview_cloud(sample_a, title="Sample capture A / サンプル撮像A")
preview_cloud(sample_b, title="Sample capture B / サンプル撮像B")


In [ ]:
def run_profile(seed: int, capture: int) -> pd.DataFrame:
    pts = synthetic_tire_points(seed, capture)
    model = fit_cylinder(pts, threshold=0.0025)
    aligned, *_ = align_points(pts, model.point_on_axis, model.axis_direction)
    features = cylindrical_features(aligned, model.radius)
    mask = slice_band(aligned, features=features, tape_width=0.02, outer_band=0.05)
    rim_mask = (np.abs(aligned[:, 0]) > 0.11) & (np.abs(features['arc']) < 0.03)
    rim_points = aligned[rim_mask][:20]
    arc = np.arctan2(rim_points[:, 1], rim_points[:, 2]) * model.radius
    rimline = fit_rimline(rim_points, arc)
    result = compute_profile(aligned, features=features, mask=mask, rimline=rimline, nbins=80)
    return result.profile

In [ ]:
sample_model = fit_cylinder(sample_a, threshold=0.0025)
sample_aligned, *_ = align_points(sample_a, sample_model.point_on_axis, sample_model.axis_direction)
sample_features = cylindrical_features(sample_aligned, sample_model.radius)
sample_mask = slice_band(sample_aligned, features=sample_features, tape_width=0.02, outer_band=0.05)
preview_cloud(
    sample_aligned,
    color=np.where(sample_mask, 1.0, 0.0),
    title="Capture A mask visualization / 撮像Aの12時帯域"
)


In [ ]:
pairs = {
    'Tire A': (run_profile(0, 0), run_profile(0, 1)),
    'Tire B': (run_profile(1, 0), run_profile(1, 1)),
}
len(pairs)

In [ ]:
def profile_metrics(df_a: pd.DataFrame, df_b: pd.DataFrame) -> dict:
    merged = pd.merge(df_a, df_b, on='x_center', suffixes=('_a', '_b'))
    diff = merged['z_mean_a'] - merged['z_mean_b']
    rmse = np.sqrt(np.mean(diff**2))
    corr = np.corrcoef(merged['z_mean_a'], merged['z_mean_b'])[0, 1]
    return {'rmse': rmse, 'corr': corr, 'diff_curve': merged[['x_center', 'z_mean_a', 'z_mean_b', 'delta_r_mean_a']]}

In [ ]:
records = []
curves = {}
for name, (a, b) in pairs.items():
    metrics = profile_metrics(a, b)
    records.append({'tire': name, 'rmse_mm': metrics['rmse'] * 1000, 'corr': metrics['corr']})
    curves[name] = pd.merge(a, b, on='x_center', suffixes=('_a', '_b'))
metrics_df = pd.DataFrame(records)
metrics_df

In [ ]:
fig, axes = plt.subplots(len(curves), 1, figsize=(8, 4 * len(curves)))
if len(curves) == 1:
    axes = [axes]
for ax, (name, merged) in zip(axes, curves.items()):
    ax.plot(merged['x_center'], merged['z_mean_a'], label='Capture 1')
    ax.plot(merged['x_center'], merged['z_mean_b'], label='Capture 2')
    ax.plot(merged['x_center'], merged['z_mean_a'] - merged['z_mean_b'], label='Diff', linestyle='--')
    ax.set_title(f'{name} repeatability')
    ax.set_xlabel('X (axial)')
    ax.set_ylabel("Z' (mm)")
    ax.grid(True, alpha=0.3)
    ax.legend()
plt.tight_layout()
plt.show()